<a href="https://colab.research.google.com/github/JoKab5/Bitcoin-prediction-project/blob/main/Ads_clustering_Joel_KabanguKabobo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Machine Learning Project**:

### **Regroupement des annonces immobilières par l'apprentissage non supervisé(Unsupervised Learning)**




> Par Joel Kabangu Kabobo









Au sein des plateformes des annonces immobilières, il n'est pas rare de constater la présence répétitive d'annonces concernant un même bien, formulées de manière distincte sur différentes plateformes et assorties de descriptions variées. En effet, de nombreux acteurs professionnels, voire des particuliers, diffusent des annonces identiques relatives à un bien immobilier sur divers portails spécialisés. Afin d'optimiser le processus de recherche et d'obtenir des données statistiques plus précises, cette étude s'est focalisée sur la détection d'annonces similaires provenant des platformes diverses. Ces annonces ont été regroupées de manière cohérente, tandis que les annonces uniques ont été traitées individuellement. Cette approche permet de distinguer les annonces uniques, considérées comme des exclusivités propres à la plateforme de publication. Notre analyse s'est concentrée sur les annonces immobilières en cours de publication pour la ville de Levallois-Perret au mois d'août 2019.

##### **Note : si vous pouvez, veillez excuter ce notebook avec le GPU pour une rapidité d'execution sinon il fonctionne aussi avec le CPU mais l'execution sera lente.**

### L'installation des bibliothèques et packages utilisées

In [2]:
!pip install hdbscan cuba sentence_transformers umap-learn transformers torchvision >/dev/null 2>/dev/nul

#### Importation des bibliothèques utilisées

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
# using plotly to visualize clusters
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
import copy
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoModel, AutoTokenizer
import torch
import torchvision
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True



### **Chargement des données**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Dataset - Ads _ Levallois-Perret - 2019-08 - export-ads-levallois-perret-2019-08-27.csv")
df.head(10)

,ID,URL,CRAWL_SOURCE,PROPERTY_TYPE,NEW_BUILD,DESCRIPTION,IMAGES,SURFACE,LAND_SURFACE,BALCONY_SURFACE,TERRACE_SURFACE,ROOM_COUNT,BEDROOM_COUNT,BATHROOM_COUNT,LUNCHROOM_COUNT,TOILET_COUNT,FURNISHED,FIREPLACE,AIR_CONDITIONING,GARDEN,SWIMMING_POOL,BALCONY,TERRACE,CELLAR,PARKING,PARKING_COUNT,HEATING_TYPES,HEATING_MODE,FLOOR,FLOOR_COUNT,CONSTRUCTION_YEAR,ELEVATOR,CARETAKER,ENERGY_CONSUMPTION,GREENHOUSE_GAS_CONSUMPTION,MARKETING_TYPE,PRICE,PRICE_M2,PRICE_EVENTS,RENTAL_EXPENSES,RENTAL_EXPENSES_INCLUDED,DEPOSIT,FEES,FEES_INCLUDED,EXCLUSIVE_MANDATE,AGENCIES_UNWANTED,OCCUPIED,DEALER_NAME,DEALER_TYPE,CITY_ID,CITY,ZIP_CODE,DEPT_CODE,PUBLICATION_START_DATE,PUBLICATION_END_DATE,LAST_CRAWL_DATE,LAST_PRICE_DECREASE_DATE
0,22c05930-0eb5-11e7-b53d-bbead8ba43fe,http://www.avendrealouer.fr/location/levallois-perret-92/b-appartement/3-pieces/loc-101-42762/fd-15565447.html,A_VENDRE_A_LOUER,APARTMENT,False,"Au rez de chaussée d'un bel immeuble récent,appartement de standing se composant d'une entrée,un grand séjour avec grande baie, cuisine entièrement équipée et aménagée (Frigidaire,four...), 2 chambres avec rangements,salle de bain en marbre avec double vasque,WC séparés avec lave -mains. Eau chaude et chauffage compris dans les charges.Parquet.Cave ,parking.Très lumineux et calme. Mise ? jour le 16/04/2017 R?f?rence annonce AVENDREALOUER : L-95-09 9-7621","[""https://cf-medias.avendrealouer.fr/image/_873908158_d.jpg"",""https://cf-medias.avendrealouer.fr/image/_873908159_d.jpg"",""https://cf-medias.avendrealouer.fr/image/_873908160_d.jpg"",""https://cf-medias.avendrealouer.fr/image/_873908161_d.jpg"",""https://cf-medias.avendrealouer.fr/image/_873908162_d.jpg""]",72.00,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0,[],COLLECTIVE,NaN,NaN,NaN,True,NaN,NaN,NaN,RENT,1979.0,27.49,"[{""price"":1979,""date"":""2017-03-22""}]",276.0,True,NaN,NaN,NaN,False,NaN,NaN,Lamirand Et Associes,AGENCY,54178039,Levallois-Perret,92300.0,92,2017-03-22T04:07:56.095,NaN,2017-04-21T18:52:35.733,NaN
1,8d092fa0-bb99-11e8-a7c9-852783b5a69d,https://www.bienici.com/annonce/ag440414-165474561,BIEN_ICI,APARTMENT,False,"Je vous propose un appartement dans la rue Collange dans un immeuble de standing, moderne et sécurisé. 49m2 loi carrez situé au premier étage avec ascenseur comprenant une entrée, un séjour-salon, une cuisine équipée et aménagée, une chambre, une salle de bain et un toilette séparé. Appartement ayant une exposition sud donnant sur le jardin baigné de lumière et bénéficiant d'une belle hauteur sous plafond, parquet, balcon-terrasse ainsi que une cave. A proximité des écoles, des commerces et transports. Les charges de copropriété sont de 1440 euros/an. Appartement sans perte de place et vue verdoyante. Prix 429.000 euros honoraires charges vendeur. Contactez Dominique GIRAUD-DESERT au 06 62 23 01 49 agissant en tant que conseillère indépendante auprès de la SAS PROPRIETES PRIVEES. CPI : 75012016000006389 Mandat 176430 Copropriété de 140 lots (). Charges annuelles : 1440 euros.","[""http://photos.ubiflow.net/440414/165474561/photos/1.jpg?20190401022226"",""http://photos.ubiflow.net/440414/165474561/photos/2.jpg?20190401022226"",""http://photos.ubiflow.net/440414/165474561/photos/3.jpg?20190401022226"",""http://photos.ubiflow.net/440414/165474561/photos/4.jpg?20190401022226"",""http://photos.ubiflow.net/440414/165474561/photos/5.jpg?20190401022226"",""http://photos.ubiflow.net/440414/165474561/photos/6.jpg?20190401022226""]",48.00,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,NaN,[],INDIVIDUAL,1.0,6.0,NaN,True,False,NaN,NaN,SALE,429000.0,8937.50,"[{""price"":452000.0,""date"":""2018-09-18""},{""price"":429000.0,""date"":""2018-09-25""}]",NaN,NaN,NaN,NaN,False,False,NaN,False,Proprietes Privees,MANDATARY,54178039,Levallois-Perret,92300.0,92,2018-09-18T11:04:44.461,NaN,2019-06-06T10:08:10.89,2018-09-25
2,44b6a5c0-3466-11e9-8213-25cc7d9bf5fc,https://www.bellesdemeures.com/annonces/vente/appartement-luxe/levallois-perr

### **Descrption et traitement des données**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          2164 non-null   object 
 1   URL                         2164 non-null   object 
 2   CRAWL_SOURCE                2164 non-null   object 
 3   PROPERTY_TYPE               2164 non-null   object 
 4   NEW_BUILD                   1973 non-null   object 
 5   DESCRIPTION                 2160 non-null   object 
 6   IMAGES                      2164 non-null   object 
 7   SURFACE                     2050 non-null   float64
 8   LAND_SURFACE                3 non-null      float64
 9   BALCONY_SURFACE             0 non-null      float64
 10  TERRACE_SURFACE             25 non-null     float64
 11  ROOM_COUNT                  1835 non-null   float64
 12  BEDROOM_COUNT               696 non-null    float64
 13  BATHROOM_COUNT              0 non

In [ ]:
#voir le nombre des lignes et des colonnes de nos données
print('Number of rows: ', df.shape[0])
print('Number of columns: ', df.shape[1])

Number of rows:  2164
Number of columns:  57


In [ ]:
#voir les nombre de données manquantes pour chaque variable
df.isnull().sum()

ID                               0
URL                              0
CRAWL_SOURCE                     0
PROPERTY_TYPE                    0
NEW_BUILD                      191
DESCRIPTION                      4
IMAGES                           0
SURFACE                        114
LAND_SURFACE                  2161
BALCONY_SURFACE               2164
TERRACE_SURFACE               2139
ROOM_COUNT                     329
BEDROOM_COUNT                 1468
BATHROOM_COUNT                2164
LUNCHROOM_COUNT               2164
TOILET_COUNT                  2164
FURNISHED                     1697
FIREPLACE                     2164
AIR_CONDITIONING              2164
GARDEN                        2150
SWIMMING_POOL                 2158
BALCONY                       2164
TERRACE                       2078
CELLAR                        2164
PARKING                          0
PARKING_COUNT                 2007
HEATING_TYPES                    0
HEATING_MODE                  1511
FLOOR               

In [ ]:
#voir les information statistique de nos données
df.describe()

,SURFACE,LAND_SURFACE,BALCONY_SURFACE,TERRACE_SURFACE,ROOM_COUNT,BEDROOM_COUNT,BATHROOM_COUNT,LUNCHROOM_COUNT,TOILET_COUNT,FIREPLACE,AIR_CONDITIONING,GARDEN,SWIMMING_POOL,BALCONY,TERRACE,CELLAR,PARKING_COUNT,FLOOR,FLOOR_COUNT,CONSTRUCTION_YEAR,ENERGY_CONSUMPTION,GREENHOUSE_GAS_CONSUMPTION,PRICE,PRICE_M2,RENTAL_EXPENSES,DEPOSIT,FEES,CITY_ID,ZIP_CODE,DEPT_CODE,PUBLICATION_END_DATE
count,2050.000000,3.000000,0.0,25.000000,1835.000000,696.000000,0.0,0.0,0.0,0.0,0.0,14.0,6.0,0.0,86.0,0.0,157.000000,660.000000,465.000000,503.000000,0.0,0.0,2.104000e+03,1991.000000,441.000000,55.000000,94.000000,2164.0,2163.0,2164.0,0.0
mean,128.136068,30.000000,NaN,31.363600,2.243597,1.847701,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.605096,2.916667,5.827957,1965.081511,NaN,NaN,4.261142e+05,5483.773420,406.687347,2988.506000,1531.935957,54178039.0,92300.0,92.0,NaN
std,423.339898,8.660254,NaN,57.467242,1.715621,1.104508,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.299694,2.123793,2.432820,37.403519,NaN,NaN,6.486603e+05,5384.760624,1329.451157,8071.172419,6269.065130,0.0,0.0,0.0,NaN
min,6.000000,25.000000,NaN,5.590000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.000000,0.000000,0.000000,1870.000000,NaN,NaN,3.300000e+01,1.780000,2.000000,70.000000,29.500000,54178039.0,92300.0,92.0,NaN
25%,36.200000,25.000000,NaN,13.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.000000,1.000000,5.000000,1930.000000,NaN,NaN,1.600000e+03,32.375000,60.000000,860.500000,378.000000,54178039.0,92300.0,92.0,NaN
50%,55.000000,25.000000,NaN,18.000000,2.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.000000,2.000000,6.000000,1976.000000,NaN,NaN,2.200000e+05,7586.210000,100.000000,1350.000000,639.670000,54178039.0,92300.0,92.0,NaN
75%,93.000000,32.500000,NaN,26.000000,3.000000,2.000000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,2.000000,4.000000,7.000000,1997.000000,NaN,NaN,6.097500e+05,10056.750000,151.000000,2450.000000,806.700000,54178039.0,92300.0,92.0,NaN
max,10287.000000,40.000000,NaN,300.000000,10.000000,8.000000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,9.000000,14.000000,19.000000,2021.000000,NaN,NaN,6.000000e+06,89000.000000,14287.330000,60000.000000,60000.000000,54178039.0,92300.0,92.0,NaN


Nous voyons que nos données contiennent 2164 lignes et 57 colonnes. Nous pouvons observer aussi d'autres informations comme la valeur moyenne, minimum, maximum,etc des variables de nos données.

Nous remarquons aussi que nos données contiennent des variables qui sont en nombre(float, int) ou en lettre(object) ou en booléenne(bol).

Nous remarquons aussi que nos données continent plus de valeures manquantes et même des variables qui ne contiennent aucune valeur que des manquantes alors nous allons supprimer ces variables là.


In [ ]:
df['BALCONY'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: BALCONY, dtype: float64

In [ ]:
#Suppression des lignes où toutes les valeurs sont null
df = df.dropna(how='all')
print('Number of rows: ', df.shape[0])

# Suppression des colonnes où toutes les valeurs sont null
nan_cols = df.columns[df.isna().all()].tolist()
df = df.drop(nan_cols, axis=1)
print('Number of columns: ', df.shape[1])

Number of rows:  2164
Number of columns:  46


In [ ]:
# Affichage des nombres de valeurs uniques pour chaque variable
for col in df.columns:
    print(col, df[col].nunique())

ID 2164
URL 2164
CRAWL_SOURCE 13
PROPERTY_TYPE 5
NEW_BUILD 2
DESCRIPTION 1914
IMAGES 1947
SURFACE 454
LAND_SURFACE 2
TERRACE_SURFACE 18
ROOM_COUNT 11
BEDROOM_COUNT 7
FURNISHED 2
GARDEN 1
SWIMMING_POOL 1
TERRACE 1
PARKING 2
PARKING_COUNT 8
HEATING_TYPES 5
HEATING_MODE 3
FLOOR 12
FLOOR_COUNT 16
CONSTRUCTION_YEAR 48
ELEVATOR 2
CARETAKER 2
MARKETING_TYPE 2
PRICE 764
PRICE_M2 997
PRICE_EVENTS 1733
RENTAL_EXPENSES 129
RENTAL_EXPENSES_INCLUDED 2
DEPOSIT 44
FEES 71
FEES_INCLUDED 2
EXCLUSIVE_MANDATE 2
AGENCIES_UNWANTED 2
OCCUPIED 2
DEALER_NAME 553
DEALER_TYPE 3
CITY_ID 1
CITY 1
ZIP_CODE 1
DEPT_CODE 1
PUBLICATION_START_DATE 2103
LAST_CRAWL_DATE 2164
LAST_PRICE_DECREASE_DATE 145


Nous pouvons bien voir nos valiriables et le nombre des valeurs uniques qu'elles contiennent.



Explorons en details certaines de ces variables pour pouvoir resoudre le problème des valeurs manquantes

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        2164 non-null   object 
 1   URL                       2164 non-null   object 
 2   CRAWL_SOURCE              2164 non-null   object 
 3   PROPERTY_TYPE             2164 non-null   object 
 4   NEW_BUILD                 1973 non-null   object 
 5   DESCRIPTION               2160 non-null   object 
 6   IMAGES                    2164 non-null   object 
 7   SURFACE                   2050 non-null   float64
 8   LAND_SURFACE              3 non-null      float64
 9   TERRACE_SURFACE           25 non-null     float64
 10  ROOM_COUNT                1835 non-null   float64
 11  BEDROOM_COUNT             696 non-null    float64
 12  FURNISHED                 467 non-null    object 
 13  GARDEN                    14 non-null     float64
 14  SWIMMING

Nous remarquons beaucoups des variables ne contiennent preque pas des valeurs. Nous sommes tenter à le supprimer mais ces variables peuvent être important pour la detection de la simiralité des annonces. Par exemple la variable "SWIMING_POOL" qui contient que 6 valeurs, cela peut être une information importante à tenir en compte car ce n'est pas toutes les mainsons qui ont des piscines alors si on en a 6 valeurs dans nos données c'est fort problable qu'il s'agisse des annonces doublons car uniquement ses annonces ont des piscines. Explorons la variable

In [ ]:
## Affichage des données des annonces avec piscine
df_swim = df[~df['SWIMMING_POOL'].isna()]
df_swim.head(6)

,ID,URL,CRAWL_SOURCE,PROPERTY_TYPE,NEW_BUILD,DESCRIPTION,IMAGES,SURFACE,LAND_SURFACE,TERRACE_SURFACE,ROOM_COUNT,BEDROOM_COUNT,FURNISHED,GARDEN,SWIMMING_POOL,TERRACE,PARKING,PARKING_COUNT,HEATING_TYPES,HEATING_MODE,FLOOR,FLOOR_COUNT,CONSTRUCTION_YEAR,ELEVATOR,CARETAKER,MARKETING_TYPE,PRICE,PRICE_M2,PRICE_EVENTS,RENTAL_EXPENSES,RENTAL_EXPENSES_INCLUDED,DEPOSIT,FEES,FEES_INCLUDED,EXCLUSIVE_MANDATE,AGENCIES_UNWANTED,OCCUPIED,DEALER_NAME,DEALER_TYPE,CITY_ID,CITY,ZIP_CODE,DEPT_CODE,PUBLICATION_START_DATE,LAST_CRAWL_DATE,LAST_PRICE_DECREASE_DATE
503,b8b93640-5f7c-11e9-adc1-59e67c527828,https://www.seloger.com/annonces/achat-de-prestige/maison/levallois-perret-92/henri-barbusse/146174921.htm,SE_LOGER,HOUSE,False,"En plein coeur de Levallois, quartier Villiers-Louise Michel, au calme, hôtel particulier d'exception au style contemporain et design. Il est composé de 6 pièces sur 3 niveaux avec ascenseur et offre de belles prestations piscine, Hammam, salle de cinéma, 3 terrasses, Patio et jardin privatif. 423 m² au sol (loi Carrez 289 m² + annexes 134 m²). Il comprend un grand séjour avec cuisine équipée, 1 suite parentale avec salle de bain, salle d'eau et dressing attenants, 4 chambres avec salles d'eau attenantes, WC séparés, buanderie. Dont 3.78 % honoraires TTC à la charge de l'acquéreur.","[""https://v.seloger.com/s/width/800/visuels/1/a/r/g/1arg135bgu63se8dhysct6m4tg7n3dox9a7427ec4.jpg"",""https://v.seloger.com/s/width/800/visuels/1/3/e/9/13e9inwuvh25c6lbl08m4tlw1ldfvzxbmyupwls9g.jpg"",""https://v.seloger.com/s/width/800/visuels/0/2/f/a/02fa4q5v3gg4th786htxi9bzwmg8of0oxxixqxxlg.jpg"",""https://v.seloger.com/s/width/800/visuels/1/e/h/m/1ehmr08n5me4wr0otz3bmfff5f54dda2fqre0qi50.jpg"",""https://v.seloger.com/s/width/800/visuels/0/z/s/r/0zsrd4oez95o0q194eadkoyiirufuz772i101nn84.jpg"",""https://v.seloger.com/s/width/800/visuels/1/q/3/q/1q3qj0al4kmrpeqy8ivynrctub7teudfy3cpf03tg.jpg"",""https://v.seloger.com/s/width/800/visuels/1/z/9/l/1z9l7pmwokld4df71oy3r78n43556nyumtd1uiwd0.jpg"",""https://v.seloger.com/s/width/800/visuels/1/y/7/y/1y7yo233yguin11p74s5rdk2eaamn6knlwy1beixg.jpg"",""https://v.seloger.com/s/width/800/visuels/1/4/h/7/14h71z1mwpp5gx0xgfcfpnr4ysfo4d9y28gub2nh0.jpg"",""https://v.seloger.com/s/width/800/visuels/1/f/f/e/1ffekezb2uup08wek8m3mcjwi3b7uut5iq68hhbo4.jpg"",""https://v.seloger.com/s/width/800/visuels/0/n/m/z/0nmzdah0d9xkqg2uucj8cqyxr2actni6cu9zpgllg.jpg"",""https://v.seloger.com/s/width/800/visuels/1/q/p/t/1qptu7cj4o98059a73xp0swbgth678edlos9u3plg.jpg"",""https://v.seloger.com/s/width/800/visuels/1/a/i/6/1ai6adehnnzv809i2swcqhn272aiqo6cm9j0fqwk4.jpg"",""https://v.seloger.com/s/width/800/visuels/0/q/q/v/0qqvm3tbxedntlfct8w2gnr93phd8bisebzkkyous.jpg"",""https://v.seloger.com/s/width/800/visuels/0/9/z/t/09ztf7gf22qwdy8ku4qralfbcfm7669zy5s1hkmz8.jpg"",""https://v.seloger.com/s/width/800/visuels/0/u/7/h/0u7hj3nfx4hb01n0oqp0otwxwij876oxtyrfyta38.jpg"",""https://v.seloger.com/s/width/800/visuels/1/g/e/9/1ge91zhh8jnnhup7f7ka6k3xxtd0l974nwri1eaoz.jpg"",""https://v.seloger.com/s/width/800/visuels/1/h/y/h/1hyhviu26jiigyq9qq15nb93i8xsk0fneowze6qpi.jpg""]",290.00,NaN,NaN,6.0,5.0,NaN,NaN,1.0,1.0,False,NaN,"[""ELECTRIC""]",INDIVIDUAL,0.0,NaN,NaN,NaN,NaN,SALE,3269000.0,11272.41,"[{""price"":3429000.0,""date"":""2019-04-15""},{""price"":3273000.0,""date"":""2019-05-10""},{""price"":3269000.0,""date"":""2019-05-11""}]",NaN,NaN,NaN,NaN,NaN,False,NaN,False,Marc Foujols Groupe Immobilier,AGENCY,54178039,Levallois-Perret,92300.0,92,2019-04-15T12:48:07.996,2019-08-08T06:51:44.022,2019-05-11
1010,ef5df090-30ac-11e9-8213-25cc7d9bf5fc,https://www.seloger.com/annonces/achat-de-prestige/maison/levallois-perret-92/villiers-l-michel/144077349.htm,SE_LOGER,HOUSE,False,"Hôtel Particulier contemporain sur 3 niveaux avec Ascenseur. 6 pièces avec piscine, hammam, salle de cinéma, 3 terrasses, balcon, Patio et jardin privatif. 423 m² au sol (Loi Carrez 289 m² + Annexe 134 m²) Grand séjour avec cuisine aménagée et équipée, 1 suite parentale avec salle de

In [ ]:
#Affichage des descriptions des annonces avec piscine

df_swim['DESCRIPTION'].apply(lambda x: print("\n new =",x))


 new = En plein coeur de Levallois, quartier Villiers-Louise Michel, au calme, hôtel particulier d'exception au style contemporain et design. Il est composé de 6 pièces sur 3 niveaux avec ascenseur et offre de belles prestations piscine, Hammam, salle de cinéma, 3 terrasses, Patio et jardin privatif. 423 m² au sol (loi Carrez 289 m² + annexes 134 m²). Il comprend un grand séjour avec cuisine équipée, 1 suite parentale avec salle de bain, salle d'eau et dressing attenants, 4 chambres avec salles d'eau attenantes, WC séparés, buanderie. Dont 3.78 % honoraires TTC à la charge de l'acquéreur.

 new = Hôtel Particulier contemporain sur 3 niveaux avec Ascenseur. 6 pièces avec piscine, hammam, salle de cinéma, 3 terrasses, balcon, Patio et jardin privatif. 423 m² au sol (Loi Carrez 289 m² + Annexe 134 m²) Grand séjour avec cuisine aménagée et équipée, 1 suite parentale avec salle de bain, sa salle d'eau et son Dressing, 4 chambres avec leur salle d'eau, WC séparés, buanderie. Très rare. Bien

503     None
1010    None
1261    None
1431    None
2115    None
2128    None
Name: DESCRIPTION, dtype: object

Par les annonces avec piscine nous remarquons clairement que l'annonce de l'index 503 et 2115 sont les mêmes car les valeurs de leur variables sont similiaire d'ou ils sont doublons et doivent être dans un même cluster dans notre regroupement.





### **Traitement de données**


Dans ce partie, nous allons soigner nos données dans un format propre pour être accepté par notre model et pour avoir des resultats souhaitable. Mais nous allons traiter la colonne la plus importante "DESCRIPTION" separement car elle contient des textes.

In [ ]:
data = df.copy()

In [ ]:
# colonnes à supprimer
col_del = ['DEPOSIT', 'FEES', 'CITY_ID', 'CITY', 'ZIP_CODE', 'DEPT_CODE', 'ID', 'DESCRIPTION', 'URL',
           'PRICE_EVENTS', 'LAST_PRICE_DECREASE_DATE', 'IMAGES', 'LAND_SURFACE', 'TERRACE_SURFACE', 'PARKING_COUNT', 'RENTAL_EXPENSES','FLOOR_COUNT', 'FLOOR']

data = df.drop(col_del, axis=1)

Nous avons supprimés les variables dont nous avons juger non importantes pour notre algorithme, et dont nous avons remarquer ayant extrement beaucoup des valeurs manquantes

In [ ]:
# Les variables categorielles
categorical_cols = ['CRAWL_SOURCE','PROPERTY_TYPE', 'NEW_BUILD', 'FURNISHED', 'PARKING', 'HEATING_MODE', 'ELEVATOR', 'CARETAKER', 'MARKETING_TYPE', 'RENTAL_EXPENSES_INCLUDED',
                    'FEES_INCLUDED', 'EXCLUSIVE_MANDATE', 'AGENCIES_UNWANTED', 'OCCUPIED', 'DEALER_TYPE', 'DEALER_NAME', 'CONSTRUCTION_YEAR', 'HEATING_TYPES']
categorical_label_encoders = {}

# La convertion des valeurs en lettre en nombre
for col in categorical_cols:
    categorical_label_encoders[col] = LabelEncoder()
    data[col] = categorical_label_encoders[col].fit_transform(data[col])

Nous avons convertir les variables qui étaient en charactère en nombre pour qu'elles soient accepter par l'algorithme. Nous avons utiliser le Label encoder car ces variables avaient tous des variables nominales.

Comme Nous avons des colonnes en date que nous trouvons importantes, nous l'avons converti en temps unix. Le temps unix, est une mesure du temps basée sur le nombre de secondes écoulées depuis le 1er janvier 1970 00:00:00 UTC, hors secondes intercalaires.

In [ ]:
date_cols = ['PUBLICATION_START_DATE', 'LAST_CRAWL_DATE']
def convert_to_timestamp(date_string):
    if isinstance(date_string, str):
        dt = pd.to_datetime(date_string)
        timestamp = dt.value // 10**9
        return timestamp
    return date_string

data[date_cols] = data[date_cols].applymap(convert_to_timestamp)

In [ ]:
data.head()

,CRAWL_SOURCE,PROPERTY_TYPE,NEW_BUILD,SURFACE,ROOM_COUNT,BEDROOM_COUNT,FURNISHED,GARDEN,SWIMMING_POOL,TERRACE,PARKING,HEATING_TYPES,HEATING_MODE,CONSTRUCTION_YEAR,ELEVATOR,CARETAKER,MARKETING_TYPE,PRICE,PRICE_M2,RENTAL_EXPENSES_INCLUDED,FEES_INCLUDED,EXCLUSIVE_MANDATE,AGENCIES_UNWANTED,OCCUPIED,DEALER_NAME,DEALER_TYPE,PUBLICATION_START_DATE,LAST_CRAWL_DATE
0,0,0,0,72.0,3.0,NaN,0,NaN,NaN,NaN,1,4,0,48,1,2,0,1979.0,27.49,1,2,0,2,2,297,0,1490155676,1492800755
1,2,0,0,48.0,2.0,1.0,2,1.0,NaN,NaN,0,4,1,48,1,0,1,429000.0,8937.50,2,0,0,2,0,422,1,1537268684,1559815690
2,1,0,0,267.0,6.0,NaN,2,NaN,NaN,NaN,0,4,3,48,2,2,1,4970000.0,18614.23,2,2,0,2,0,423,0,1550594943,1560414134
3,11,4,0,50.0,0.0,NaN,2,NaN,NaN,NaN,0,4,3,48,2,2,0,89.0,1.78,2,2,0,2,2,269,0,1530890339,1560854407
4,1,2,0,330.0,8.0,NaN,2,NaN,NaN,NaN,0,4,3,48,2,2,1,2790000.0,8454.55,2,2,0,2,0,164,0,1554743540,1560524538


#### **Traitement des valeurs manquantes**

Nous avons resolu le problèmes des valeurs manquantes des deux facons:
* Pour les variables "piscine", "jardin", et "terrace", nous avons remplacer les valeurs manquantes par 0 car nous pensons que ses informations ne soit pas ecrites à l'annonce ce que le bien immobilier n'a pas une piscine ou un jardin ou une terrace d'où pour ce bien là on mettra ces variables à 0

* Pour certaines variables qui sont déjà en nombre et que nous trouvons importantes mais qui manquent des valeurs, nous avons remplacé les valeurs manquantes par la median de la variable. Car la mediane represente bien la valeur du milieu des observations.

In [ ]:
col_fill = ['SWIMMING_POOL', 'GARDEN', 'TERRACE']
col_fill_med = ['SURFACE', 'ROOM_COUNT', 'BEDROOM_COUNT', 'PRICE_M2', 'PRICE']

for col in col_fill:
    data[col].fillna(0, inplace=True)

for col in col_fill_med:
    data[col].fillna(data[col].median(), inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CRAWL_SOURCE              2164 non-null   int64  
 1   PROPERTY_TYPE             2164 non-null   int64  
 2   NEW_BUILD                 2164 non-null   int64  
 3   SURFACE                   2164 non-null   float64
 4   ROOM_COUNT                2164 non-null   float64
 5   BEDROOM_COUNT             2164 non-null   float64
 6   FURNISHED                 2164 non-null   int64  
 7   GARDEN                    2164 non-null   float64
 8   SWIMMING_POOL             2164 non-null   float64
 9   TERRACE                   2164 non-null   float64
 10  PARKING                   2164 non-null   int64  
 11  HEATING_TYPES             2164 non-null   int64  
 12  HEATING_MODE              2164 non-null   int64  
 13  CONSTRUCTION_YEAR         2164 non-null   int64  
 14  ELEVATOR

#### **Traitement de la colonne "DESCRIPTION" avec NLTK**

En utilisant la bibliothèque NLTK, nous avons commencé nous tirer quelques informations de la colonne "Description" telles que le nombre de token des mots pour chaque ligne de cette variable, le nombre des mots vides(stop words) et le nombre de mots clé regroupant les adjectifs, noms, verbes, etc.

Nous avons pas voulu utiliser d'autre informations comme la longeur des mots ou des phrases et d'autre operation de Nltk car nous pensons que ça sera pas utile car par exemple deux annonces peuvent avoir une même longeur ou les mêmes nombre d'adjectifs et verbes mais sont totalement deux differentes annonces.


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

df_ = pd.DataFrame()

STOP_WORDS = set(stopwords.words('french'))

def extract_keywords(text):
    tokens = word_tokenize(str(text))
    pos_tags = pos_tag(tokens)

    # Define relevant parts of speech for keywords (e.g., nouns, adjectives)
    relevant_pos = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS']

    keywords = [word for word, pos in pos_tags if pos in relevant_pos]

    return keywords

def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('french'))
    words = str(text).split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count


# This function applies all the above preprocessing functions on a text feature

def run(df: pd.DataFrame) -> pd.DataFrame:

    df_["desc_word_len"] = df["DESCRIPTION"].apply(lambda x: len(word_tokenize(str(x))))


    df_["desc_word_cnt"] = df["DESCRIPTION"].apply(lambda x: count_stopwords(x))

    # Automatically extract keywords from the prompt
    df_['desc_keywords'] = df['DESCRIPTION'].apply(lambda x: len(extract_keywords(x)))


    return df_

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning:

The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.



In [ ]:
result = run(df)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   desc_word_len  2164 non-null   int64
 1   desc_word_cnt  2164 non-null   int64
 2   desc_keywords  2164 non-null   int64
dtypes: int64(3)
memory usage: 50.8 KB


In [ ]:
# Melange de nos données numeriques et les variables des premieres traitement de la colonne DESCRIPTION
data =data.join(result)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CRAWL_SOURCE              2164 non-null   int64  
 1   PROPERTY_TYPE             2164 non-null   int64  
 2   NEW_BUILD                 2164 non-null   int64  
 3   SURFACE                   2164 non-null   float64
 4   ROOM_COUNT                2164 non-null   float64
 5   BEDROOM_COUNT             2164 non-null   float64
 6   FURNISHED                 2164 non-null   int64  
 7   GARDEN                    2164 non-null   float64
 8   SWIMMING_POOL             2164 non-null   float64
 9   TERRACE                   2164 non-null   float64
 10  PARKING                   2164 non-null   int64  
 11  HEATING_TYPES             2164 non-null   int64  
 12  HEATING_MODE              2164 non-null   int64  
 13  CONSTRUCTION_YEAR         2164 non-null   int64  
 14  ELEVATOR

### **Mise à l'échelle des données**


Nous avons mis à l'échelle de nos données numeriques pour une comparaison équitable des caractéristiques car si les caractéristiques de nos données ont des échelles très différentes, certaines caractéristiques pourraient dominer l'apprentissage du modèle par rapport aux autres.

In [ ]:
# scale numeric columns
numeric_cols = [col for col in data.columns]

from sklearn.compose import ColumnTransformer

# scale numeric columns and categorical columns
from sklearn.preprocessing import StandardScaler
scaler_transformer = ColumnTransformer(
    transformers=[
        ('Scaler', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough'
)
new_cols = numeric_cols + [col for col in data.columns if col not in numeric_cols]
df_scaled = pd.DataFrame(scaler_transformer.fit_transform(data), columns=new_cols)

In [ ]:
df.shape, df_scaled.shape

((2164, 46), (2164, 31))

In [ ]:
#Affichage de nos données mises en echelle
df_scaled.head(10)

,CRAWL_SOURCE,PROPERTY_TYPE,NEW_BUILD,SURFACE,ROOM_COUNT,BEDROOM_COUNT,FURNISHED,GARDEN,SWIMMING_POOL,TERRACE,PARKING,HEATING_TYPES,HEATING_MODE,CONSTRUCTION_YEAR,ELEVATOR,CARETAKER,MARKETING_TYPE,PRICE,PRICE_M2,RENTAL_EXPENSES_INCLUDED,FEES_INCLUDED,EXCLUSIVE_MANDATE,AGENCIES_UNWANTED,OCCUPIED,DEALER_NAME,DEALER_TYPE,PUBLICATION_START_DATE,LAST_CRAWL_DATE,desc_word_len,desc_word_cnt,desc_keywords
0,-1.662557,-0.693198,-0.378610,-0.126821,0.501598,-0.368106,-2.465859,-0.080695,-0.052729,-0.203435,3.575394,0.381856,-1.998750,0.485786,-1.473941,0.336513,-1.135758,-0.653427,-1.082623,-1.501738,0.532470,-0.289181,0.317393,1.119918,0.292692,-0.390246,-4.227868,-7.235731,-0.417299,-0.621698,-0.389521
1,-1.168934,-0.693198,-0.378610,-0.185036,-0.130585,-0.368106,0.492898,12.392394,-0.052729,-0.203435,-0.279689,0.381856,-1.120161,0.485786,-1.473941,-3.860686,0.880469,0.013431,0.632448,0.605759,-2.095272,-0.289181,0.317393,-0.901352,1.102483,1.262380,-1.196145,-0.422071,0.697967,0.835014,0.682765
2,-1.415745,-0.693198,-0.378610,0.346180,2.398147,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,0.880469,7.104895,2.495104,0.605759,0.532470,-0.289181,0.317393,-0.901352,1.108962,-0.390246,-0.338600,-0.361224,0.116772,0.457348,0.173429
3,1.052371,1.620849,-0.378610,-0.180185,-1.394951,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,-1.135758,-0.656379,-1.087572,0.605759,0.532470,-0.289181,0.317393,1.119918,0.111299,-0.390246,-1.606592,-0.316460,2.520091,3.640532,2.478844
4,-1.415745,0.463825,-0.378610,0.498996,3.662513,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,0.880469,3.700492,0.539486,0.605759,0.532470,-0.289181,0.317393,-0.901352,-0.568925,-0.390246,-0.071638,-0.349999,1.216330,1.698250,1.486979
5,-0.922122,-0.693198,-0.378610,0.044818,1.765964,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,0.880469,1.561029,0.826719,0.605759,-2.095272,-0.289181,0.317393,-0.901352,-1.067757,-0.390246,-2.192521,-0.121573,1.263453,0.996871,0.897222
6,1.052371,1.620849,-0.378610,-0.148651,-1.394951,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,-1.135758,-0.653800,-1.082598,0.605759,0.532470,-0.289181,0.317393,1.119918,-0.983538,-0.390246,-4.028304,-0.121575,-0.605795,-0.999364,-0.603979
7,-0.922122,1.042337,2.999998,-0.262657,-0.130585,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,-1.135758,-0.656424,-1.087193,-1.501738,0.532470,-0.289181,0.317393,1.119918,1.640184,-0.390246,-1.821135,-0.126086,0.116772,0.025730,0.334272
8,-0.675310,1.042337,2.999998,-0.274785,-0.130585,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,-1.135758,-0.656362,-1.086165,-1.501738,0.532470,-0.289181,-3.204856,1.119918,1.685533,2.915005,-0.214603,-0.096700,-1.328361,-1.269125,-1.327772
9,-0.675310,-0.693198,-0.378610,-0.163205,-0.130585,-0.368106,0.492898,-0.080695,-0.052729,-0.203435,-0.279689,0.381856,0.637018,0.485786,0.561546,0.336513,0.880469,0.233624,0.836967,0.605759,0.532470,-0.289181,0.317393,-0.901352,0.065951,-0.390246,0.295300,-0.094058,0.163896,0.619205,0.414693


Nous avons maintenant nos variables importantes en proper format et sans valeurs manquantes.

Nous procedons maintenant au traitement approfondie de la variable "Description"

### **Traitement de la colonne "DESCRIPTION" avec l'embedding**

La variabele "DESCRIPTION" est la variable qui aura la grande impacte dans l'entrainement de notre algorithme.

Pour mieux traiter la variable qui contiennent les description, nous devons pas ce filier au nombre des mots ou des mots vides(stopword) ou d'autres methodes des bases mais nous devons étudier et trouver les valeurs de similarité entre les descriptions des annonces. Pour cela nous avons utilisées deux methodes et faire de prediction avec chacune des méthodes pour evaluer notre regroupement des annonces similaires:

1. Méthode avec l'encodage du text par le modèle d'integration(embedding):
2. Méthode avec la similarité cosinus(cosine similarity) du text

Nous avons utilisés pour ce deux méthodes le model pre-entrainé "**Sentence-CamemBERT-Large**"
Sentence-CamemBERT-Large est le modèle d'intégration pour le français développé par La Javaness. Le but de ce modèle d'intégration est de représenter le contenu et la sémantique d'une phrase française dans un vecteur mathématique qui lui permet de comprendre la signification du texte au-delà des mots individuels dans les requêtes et les documents, offrant ainsi une recherche sémantique puissante.
Vous pouvez retrouver ce modèle au site d'hugging face


#### 1)Traitement des "descriptions" avec l'integration(embedding)


L'idée derrière l'embedding est de projeter des données textuelles dans un espace vectoriel où des similarités sémantiques sont préservées. En d'autres termes, des éléments similaires auront des représentations vectorielles similaires. Cette technique est largement utilisée pour résoudre des tâches NLP telles que la classification de texte, la recherche d'informations, la traduction automatique, la recommandation de contenu, etc.

Les embeddings sont souvent créés en utilisant des modèles d'apprentissage profond, tels que des réseaux de neurones, qui apprennent à représenter les données textuelles de manière à capturer les relations sémantiques entre les mots. Certains des embeddings les plus populaires incluent Word2Vec, GloVe (Global Vectors for Word Representation) et les embeddings issus de modèles de langage pré-entraînés comme BERT, GPT-3, etc.

Nous utilisons un modèle d'intégration(embedding) au lieu de tfidf, word2vec pour gérer efficacement d'autres annonces ne figurant pas dans les données d'entrainement de notre algorithme, pour avoir des meilleures performances, et la gestion des mots hors vocabulaire.

Le modèle d'intégration prend un maximum de 512 jetons et le code dans un vecteur de 768. Nous utilisons un grand modèle pour de meilleures performances, mais il existe des alternatives plus rapides pour des résultats légèrement moins bons.

Nous

In [ ]:

from sentence_transformers import SentenceTransformer, util
import umap


# utilisation du GPU si disponible pour l'execution rapide du modèle mais si ce n'est pas le cas le modèle s'excutera au CPU comme d'habitude

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('dangvantuan/sentence-camembert-large')
model.to(device)
if device == 'cuda':
    model = model.half()


# Ajouter la colonne descritpion en specifiant que l'ajout de 500 characteres chaque fois car le modèle attribut que 514 jetons à la fois
max_desc_len = 500
description_embeddings = df['DESCRIPTION'].apply(lambda x: f"query: {str(x)[:max_desc_len]}").tolist()
description_embeddings = df['DESCRIPTION'].apply(lambda x: str(x)[:max_desc_len]).tolist()
description_embeddings = model.encode(description_embeddings, show_progress_bar=True, convert_to_tensor=False)


# faire une réduction de dimensionnalité en utilisant umap
desc_reducer = umap.UMAP(n_neighbors=2, n_components=100, metric='euclidean', verbose=True)

description_embeddings = desc_reducer.fit_transform(description_embeddings)

# faire un dataframe avec l'encodage des descritpions
df_embeddings = pd.DataFrame(description_embeddings)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.25k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

Batches:   0%|          | 0/68 [00:00<?, ?it/s]

UMAP(n_components=100, n_neighbors=2, verbose=True)
Wed Dec 13 19:05:10 2023 Construct fuzzy simplicial set
Wed Dec 13 19:05:15 2023 Finding Nearest Neighbors
Wed Dec 13 19:05:18 2023 Finished Nearest Neighbor Search
Wed Dec 13 19:05:22 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Wed Dec 13 19:05:30 2023 Finished embedding


In [ ]:
# voir les nombres des lignes et variables qu'a produit notre modèle
df_embeddings.shape

(2164, 100)

## **Apprentisage du modèle**

Nous avons remarqué que dans nos données, nous avons pas une colonne cible par laquelle nous pouvons dire si nos données sont similaire ou pas. Mais nous devons chercher les éléments de similarité entre les annonces regrouper les annonces similaire, il s'agit ici d'un problème de l'apprentissage non supervisé(Unserpervised Learning) et de l'approche "clustering" de l'apprentisage non supervisé.




La méthode du regroupement(clustering) est une approche consitant à trouver les éléments de similarité entre les données afin de le regrouper ensemble.
Utilisant cette technique, nous serons à mesure de regrouper les annonces similaires ensemble.

### Train et test split

Nous melangeons nos données numeriques traités et nos données resultantes du traitement de la variable "DESCRIPTION"

Nous divisons l'ensemble de données en deux parties, à savoir l'ensemble d'entraînement et de test avec un rapport de 80 % pour l'ensemble d'entraînement et de 20 % pour l'ensemble de test.

Nous faisons cette division pour éviter le sur-ajustement(overfitting), c'est pourquoi nous divisons notre ensemble de données en divisions d'entraînement et de test, ce qui nous donne une meilleure idée de la façon dont notre algorithme va fonctionné pendant la phase d'entraînement et de test.

Nous utilisons la méthode train_test_split() de sklearn pour faire cette division et nous initialisons le random_seed à 42.

le random_seed est utilisé pour initialiser le générateur de nombres aléatoires interne, qui décidera de la division des données en indices de train et de test dans votre cas.

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2

df_with_all_features = pd.concat([df_scaled, df_embeddings], axis=1)
df_with_all_features['CRAWL_SOURCE_original'] = df['CRAWL_SOURCE']
df_with_all_features = pd.concat([df_with_all_features, df['DESCRIPTION']], axis=1)
X_train_full, X_test_full = train_test_split(df_with_all_features, test_size=test_size, random_state=42)
X_train, X_test = X_train_full.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore'), X_test_full.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore')

In [ ]:
# voir la forme de nos données apres divisions
X_train.shape, X_test.shape, X_train_full.shape, X_test_full.shape

((1731, 131), (433, 131), (1731, 133), (433, 133))

### **L'aglorithme de regroupement(clustering)**

Il existe plusieurs algorithme de regroupement dans l'apprentissage automatique mais pour notre problème, nous avons choisi d'utiliser l'algorithme HDBSCAN.

HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) est un algorithme de regroupement (clustering) de données basé sur la densité. Il a été développé pour identifier des clusters dans des ensembles de données complexes et bruités, en particulier lorsque la forme, la taille et la densité des clusters peuvent varier.

HDBSCAN est une extension de l'algorithme DBSCAN (Density-Based Spatial Clustering of Applications with Noise), qui est également basé sur la densité. Cependant, HDBSCAN introduit une approche hiérarchique pour le regroupement, ce qui signifie qu'il peut identifier des clusters de différentes densités et hiérarchies dans les données. Il utilise un arbre de regroupement (clustering tree) pour représenter la hiérarchie des clusters.

Les avantages de HDBSCAN incluent sa capacité à détecter automatiquement le nombre de clusters dans les données, sa résistance au bruit et sa capacité à gérer des données de formes et de densités variées. Il est largement utilisé dans l'exploration de données, la visualisation de données et l'analyse de clusters dans divers domaines, tels que l'apprentissage automatique, la bioinformatique, la géolocalisation, la vision par ordinateur, etc.

In [ ]:
import hdbscan
seed_everything(42)
#Nous avons unitialisier l'algorithm en specifiant certaines paramètres et en laissant les autres par defaut.
# min_cluster_size: Le nombre minimum d'échantillons dans un groupe pour que ce groupe soit considéré comme une grappe ; les groupements plus petits que cette taille seront laissés comme bruit.
# alpha : Un paramètre d'échelle de distance utilisé dans une liaison simple robuste.
# min_samples : Le nombre d'échantillons dans un quartier pour qu'un point soit considéré comme un point central. Cela inclut le point lui-même

clusterer = hdbscan.HDBSCAN(min_cluster_size=2,
                            min_samples=1, # min_cluster_size by default
                            alpha=5.0, # 1.0 by default
                            prediction_data=True
                            )
# entrainement du modèle
clusterer.fit(X_train)


HDBSCAN(alpha=5.0, min_cluster_size=2, min_samples=1, prediction_data=True)

### Visualition de nos resultat de regroupement


Nous avons utilisé la méthode de reduction de dimensionalité "umap" pour nous permettre de visualiser nos regroupement sur une dimension 2D que sur la dimensionalité de nos variables.

In [ ]:
# Utilisation d'Umap
seed_everything(42)
reducer = umap.UMAP(n_neighbors=6, n_components=2)
X_train_pca = reducer.fit_transform(X_train)

In [ ]:
# plot clusters

# tronquer la description à 100 caractères
X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION'].str[:600]
X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION_wrapped'].str.wrap(100)
X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION_wrapped'].apply(lambda x: str(x).replace('\n', '<br>'))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X_train_pca[:, 0],
    y=X_train_pca[:, 1],
    mode='markers',
    marker=dict(
        size=5,
        color=clusterer.labels_,

    ),
    text=X_train.index,
    customdata=X_train_full['DESCRIPTION_wrapped'],
    hovertemplate='Description: %{customdata}<br>Index: %{text}<br>Cluster: %{marker.color}<extra></extra>'
))

# increase size of the plot
fig.update_layout(
    title='Clusters',
    xaxis_title='PCA 1',
    yaxis_title='PCA 2',
    legend_title='Cluster',
    font=dict(
        family='Courier New, monospace',
        size=18,
        color='#7f7f7f'
    ),
    width=1200,
    height=800
)
X_train_full.drop('DESCRIPTION_wrapped', axis=1, inplace=True)

fig.show()

Nous pouvont aller au dessus de chaque point de notre visualisation et nos verons la description et le cluster qu'appartient l'annonce.

Comme la visualisation n'est pas assez visible, nous allons l'affiche sous forme tabulaire

### Ajout de la colonne de regroupement


Nous avons ajouter la colonne "cluster" de regroupement des annonces dans nos données d'entrainement pour nous permettre de bien voir les annonces en fonction de regoupement et pour pouvoir evaluer notre algorithme

In [ ]:
def add_cluster_column(clusterer, X_train_):
    X_train = copy.deepcopy(X_train_).reset_index(drop=True)
    labels = clusterer.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Number of outliers/unique ads: %d' % list(labels).count(-1))

    # Affichage du nombre maximun du cluster ayant le plus des annonces
    print('Max cluster: %d' % np.sort(np.unique(labels, return_counts=True)[1])[::-1][1]) # ignore -1, car -1 represente les valeurs aberrantes d'ou des annonces considerés uniques

    # Affichage de la moyenne des annonces dans des clusters
    print('Mean cluster: %d' % np.unique(labels, return_counts=True)[1].mean())

    # Affichage du nombre minimum
    print('Min cluster: %d' % np.unique(labels, return_counts=True)[1].min())


    # Revoir les réels valeurs de nos donées avant la mise en echelle
    _X_train = pd.DataFrame(scaler_transformer.named_transformers_['Scaler'].inverse_transform(X_train[numeric_cols]), columns=numeric_cols)
    X_train[numeric_cols] = _X_train[numeric_cols]


    # ajout de la colonne regroupement
    X_train['CLUSTER'] = labels

    # créez une colonne contenant la distance par rapport au centre de gravité du cluster. -1 distance signifie que le point est une valeur aberrante
    X_train['CLUSTER_DISTANCE'] = clusterer.outlier_scores_
    return X_train


X_train_full_clustered = add_cluster_column(clusterer, X_train_full)

Estimated number of clusters: 226
Number of outliers/unique ads: 1131
Max cluster: 15
Mean cluster: 7
Min cluster: 2


Nous voyons que notre modèle a reussi de faire 226 groupements des annonces similaires et à classer 1131 annonces comme étant uniques.

Nous pouvons aussi remarquer que le plus gros groupe a 15 annonces similaires, le plus petit groupe en a 2 et la moyenne des annonces dans les groupes est de 7


In [ ]:
#cheque les platformes source des annonces
df['CRAWL_SOURCE'].value_counts()

SE_LOGER                877
LE_BON_COIN             246
A_VENDRE_A_LOUER        208
EXPLORIMMO              196
MEILLEURS_AGENTS        193
BIEN_ICI                178
LOGIC_IMMO              137
PAP                      49
PARU_VENDU               31
PROPRIETES_DE_FRANCE     21
BELLES_DEMEURES          19
LUX_RESIDENCE             7
SURFACE_PRIVEE            2
Name: CRAWL_SOURCE, dtype: int64

Nous avons chequer les valeurs uniques de sources des annonces c’est 12, ce qui pourrait dire normalement qu’on devait pas avoir un cluster avec plus des 12 annonces similaire mais cela devrait être le cas si dans chaque platform par exemple le bon coin chaque utilisateur peut creer q’un seul compte et publier l’annonce du bien qu'une fois mais defois la personne ou plusieurs personnes peut publier l’annonce du même bien immobilier 2 fois voir meme 3 fois si les membres de sa famille le publie aussi ou bien si lui meme fait une deuxième annonce du meme bien en changeant ou en rajoutant quelque mots à la description. Notre modèle est capablede detecter ses genres d’annonces, c’est pourquoi nous avons de cluster avec plus de 12 annonces indiquant que c’est le même bien immobilier mais plusieurs fois publiés.

### **Afficher nos resultats en fonction des clusters**

In [ ]:
# voir nos resultas en groupant par les regroupements des annoces
X_train_full_clustered.sort_values(by='CLUSTER', ascending=False)[['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head(50)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
658,225,0.000000,URGENT A LOUER PARKING 96 RUE ARISTIDE BRIAN\npremier sous sol avec bip pour petite voiture,110.0,12.0,LE_BON_COIN
911,225,0.390353,"Nous mettons en location une place de parking en parfait état situé au 2ème sous-sol d'un immeuble moderne à proximité du métro Pont de Levallois (rue Marius Aufan, proche rue Paul Vaillant-Couturier).\n\nLarge rampe d'accès.\nTaille standard, pour tout type de véhicule.\nRésidence sécurisée, porte automatisée (bip).\n\nDisponible 1er septembre.",100.0,12.0,LE_BON_COIN
1475,225,0.394520,LOUE place de parking dans résidence de standing sécurisée.\nAccès facile avec badge.\nLoyer mensuel : 100 € toutes charges comprises.\nDisponibilité immédiate.,100.0,11.0,LE_BON_COIN
551,225,0.407556,"Bonjour,\n\nParticulier loue place de stationnement dans le parking souterrain d'une résidence proche de la station de métro Pont de Levallois.\n\nDisponible immédiatement.",75.0,11.0,LE_BON_COIN
654,225,0.000000,place de parking au 77 rue P.V. Couturier \nen sous-sol avec accès par télécommande \nconvient pour une citadine ou une compacte,100.0,10.0,LE_BON_COIN
27,225,0.000000,DISPONIBLE FIN AOUT\nLoue grande place de parking rue camille pelletan.\nAccessible via une rampe très très large.\nEtage -2.,110.0,11.0,LE_BON_COIN
1595,225,0.000000,"Location d'une place de parking spacieuse dans un immeuble standing.\n(1er sous-sol, facile d'accès avec bip, vidéo-surveillance.)\nSitué à l'angle rue Chaptal et rue Kléber.\nDisponible de suite.",110.0,12.0,LE_BON_COIN
753,224,0.000000,"BOXE FERME EN 1ER SOUS SOL\n\nDans une résidence sécurisée,BOX SIMPLE, situé au 11 Greffulhe, à proximité du métro ligne 3: Pont de Levallois-Bécon\nRéférence annonce : LEVALBSL\nNon soumis au DPE",150.0,12.0,LE_BON_COIN
492,224,0.000000,"PLACE DE STATIONNEMENT EN 1ER SOUS SOL\n\nDans une résidence sécurisée, PLACE DE STATIONNEMENT située au 1er sous-sol, située au 11 rue Greffulhe, à proximité du métro ligne 3 : Pont de Levallois-Bécon.\nRéférence annonce : LEVALPSL\nNon soumis au DPE",110.0,12.0,LE_BON_COIN
780,223,0.000000,"2 BIS, RUE BAUDIN - METRO PONT DE LEVALLOIS - EMPLACEMENT DE PARKING AU 2° SOUS SOL - ACCES PAR RAMPE. LOYER 150 EUROS. HONORAIRES A LA CHARGE DU LOCATAIRE 250 EUROS TTC. DEPOT DE GARANTIE 300 EUROS .",150.0,10.0,EXPLORIMMO


Nous pouvons voir que notre algorithme a trés bien bien regroupé les annoces similaire venant de different platform dans des groupes identitiques mais qui a aussi fait un peu de confusion dans d'autres groupes car les structures des descriptions de ces annonces parraissent similaire.

In [ ]:
# Affiche des 10 premiers clusters ayant des plus d'elements
print(X_train_full_clustered['CLUSTER'].value_counts()[:10])

-1      1131
 154      15
 188      11
 175       9
 178       8
 113       7
 225       7
 182       6
 140       6
 220       6
Name: CLUSTER, dtype: int64


In [ ]:
# Affichage des données du cluster le plus gros
X_train_full_clustered[X_train_full_clustered['CLUSTER'].value_counts().index[1] == X_train_full_clustered['CLUSTER']][['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head(50)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
22,154,0.977470,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",947000.0,85.00,SE_LOGER
196,154,0.900885,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",516000.0,44.92,SE_LOGER
207,154,0.899579,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",661000.0,64.05,SE_LOGER
288,154,0.981888,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",1359000.0,111.86,SE_LOGER
441,154,0.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",687000.0,61.85,SE_LOGER
534,154,0.991474,"LEVALLOIS Un appartement traversant de 3 pièces d'environ 65m² dans une résidence sécurisée avec ascenceur et bien entretenu. L'appartement se compose d'une entrée avec rangement, d'un grand séjour exposé sud-ouest, une cuisine indépendante entièrement aménagée, de deux chambres sur jardin, d'une salle de bains et des wc séparés. Nombreux rangements. Calme et ensoleillé. Une cave complète ce bien. Charges annuelles : 3552 € Soumis au statut de la copropriété Prix de vente : 620.000 € dont 2.48% d'honoraires à la charge de l'acquéreur. Building Partners, fort d'une expertise de plus de 20 ans, est spécialisé dans la transaction de biens ""Familiaux de Standing"" et bénéficie d'implantations de choix dans l'ouest parisien. C’est avec plus de 450 appartements, maisons et hôtels particuliers que Building Partners offre le plus large choix de biens à la vente.

Nous pouvons aussi remarqué qu'avec le gros cluster notre modèle à reussi à dectecter les annonces similaire venant de plusieurs platforme differentes.

In [ ]:
# Affichage des données du cluster le plus petit
X_train_full_clustered[X_train_full_clustered['CLUSTER'].value_counts().index[-1] == X_train_full_clustered['CLUSTER']][['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head()

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
1643,54,0.0,"LEVALLOIS-PERRET 1 pièce 22.65 m²\n\nA deux pas du centre commercial So Ouest dans résidence récente avec gardien & ascenseur, studio en bon état, lumineux, vue dégagée, en tout collectif entrée, pièce principale avec coin cuisinette, salle d'eau avec wc, placard\nAssurance Multirisques Privilège *: 10.01 € / mois\n*Services facultatifs, voir conditions en agences\nRéférence annonce : 1558980\nHonoraires à la charge du locataire : 339 € TTC dont 67 € pour l'état des lieux\nDépôt de garantie : 590 €\nMontant des charges : 113 € / mois\nModalité de récupération des charges locatives : provision avec régularisation annuelle",703.0,55.0,LE_BON_COIN
1651,54,0.0,"LEVALLOIS-PERRET 3 pièces 50.65 m²\n\nen plein centre ville, dans un quartier commerçant, immeuble ancien et sécurisé un 3 pièces bien aménagé avec belle cuisine et cheminée. à visiter Entrée, séjour, cuisine américaine, deux Salle de bains, wc et cave.\nAssurance Multirisques Privilège *: 16.69 € / mois\n*Services facultatifs, voir conditions en agences\nRéférence annonce : 1552939\nHonoraires à la charge du locataire : 759 € TTC dont 151 € pour l'état des lieux\nDépôt de garantie : 1211 €\nMontant des charges : 110 € / mois\nModalité de récupération des charges locatives : provision avec régularisation annuelle",1321.0,55.0,LE_BON_COIN


### Prediction de nos donées de test

In [ ]:
def pred_cluster(clusterer, df, scaled_cols):
    df = copy.deepcopy(df).reset_index(drop=True)
    labels, strengths = hdbscan.approximate_predict(clusterer, df.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore'))

    # undo scaling
    _df = pd.DataFrame(
    scaler_transformer.transformers_[0][1].inverse_transform(df[numeric_cols]),
    columns=numeric_cols
)

    df[scaled_cols] = _df[scaled_cols] # update X_train with unscaled values
    # add cluster column
    df['CLUSTER'] = labels
    # make a column that contains distance from the cluster centroid. -1 distance means that the point is an outlier
    df['CLUSTER_DISTANCE'] = strengths
    return df
X_test_full_clustered = pred_cluster(clusterer, X_test_full, numeric_cols)

### Affichage de resultats de nos predicitions de notre 1er modèle avec les données de test

In [ ]:
X_test_full_clustered.sort_values(by='CLUSTER', ascending=False)[['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'CRAWL_SOURCE_original']].head(30)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,CRAWL_SOURCE_original
335,222,0.699928,"Cession de bail 92300 Levallois-Perret Le groupe POINT DE VENTE vous propose à proximité du métro Pont de Levallois - Bécon : un commerce idéal pour Restauration sans nuisance ou alimentaire d'une surface totale de 44 m² disposant d'une façade de 6 m - Au rez de chaussée, une surface de 44 m². Prix : 55 000 € Loyer : 1 908, 50 € /HC /HT /MOIS Honoraires : 7 700 € H.T Ref annonce : P1-27014 Agence : Point de vente 01 77 39 27 91 Conseil en immobilier commercial, Point de vente vous accompagne dans vos recherches de fonds de commerce, droit au bail, locations pures, murs de boutiques. Mise à jour le 19/08/2019 Demander des infos sur ce bien Référence annonce AVENDREALOUER : P1-27014",55000.0,A_VENDRE_A_LOUER
281,219,0.847921,"Le groupe POINT DE VENTE vous propose un local commercial en angle d'une surface de 52 m² au rez-de-chaussée, Cave accessible par une trappe intérieure d'environ 45 m². Le bien se situe dans un environnement très commerçant à quelques minutes à pied du métro Anatole France et des enseignes connues telles que La Poste, l'Atelier du Sourcil, Monoprix, Zara Home.. Restauration rapide sur dossier. Honoraires TTC en sus : 15500.00 €.",4509.0,SE_LOGER
75,216,0.752165,"Le Groupe Babylone vous propose un local commercial en très bon état d'usage, avec une grande vitrine donnant sur deux angles d'environ 20 mètres linéaires, situé dans le centre de Levallois-Perret. Cession: 100 000 euros.",100000.0,SE_LOGER
149,216,0.693393,"L'agence FLAGSHIP vous propose une boutique d'environ 97 m² composée d'un rez- de-chaussée de 67 m² et d'un sous-sol de 30m² environ, directement accessible par un escalier intérieur. Activités idéales: Tous commerces sauf nuissances, possibilité restauration rapide.",55000.0,SE_LOGER
182,215,0.762483,"Dans une résidence de standing , et proche du métro Pont de Levallois, un emplacement de parking en sous-sol. Mise à jour le 02/06/2019 Demander des infos sur ce bien Référence annonce AVENDREALOUER : 537",90.0,A_VENDRE_A_LOUER
85,213,0.450562,"Surfaces de bureaux à Levallois Perret, dans un immeuble de standing. La surface est meublée, cloisonnée et climatisée. Espace lumineux, à proximité des transports en commun.",4785.0,SE_LOGER
109,213,0.414517,"Dans un immeuble de qualité en plein coeur de Levallois, à toute proximité du métro et des commerces, nous vous proposons des surfaces de bureaux climatisées, cloisonnées et en open space.",6804.0,SE_LOGER
212,211,0.675035,"92300 LEVALLOIS PERRET - VRAI COUP DE COEUR : Je vous propose ce restaurant de restauration rapide avec extraction dans un quartier d'affaires. Ce restaurant d'une superficie de 50m2, dispose de 6/8 places assises et terrasse. Matériel très récent. Bail échéance en 2028. Aucun travaux à prévoir! Loyer mensuel 1990 euros. A découvrir! Cession à 79900 euros honoraires inclus à la charge du vendeur. Affaire à saisir rapidement! Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 187446",79900.0,BIEN_ICI
95,210,1.000000,"Je vous propose un restaurant d'angle de 50 m2 en plein c?ur de Levallois Perret.Il y a 38 places assises à l'intérieur et une vingtaine en terrasse, une cuisine aménagée avec une hotte à charbon.Actuellement cuisine Française.La clientèle est une clientèle bureaux le midi et résident le soir avec une grande demande de livraison. Affaire à saisir rapidement compte tenu de son emplacement et de son potentiel de développement. Loyer:2850 Euros par mois charges comprises.M. Yankel KOSKAS 0637729002 / 0180916809poste8326 commerces.75017@maxihome.email Agent commercial indépendant du réseau national clairimmo Maxihome N° RSAC : 815 116 13200011 PRIUM - Ville du greffe : PARIS",150000.0,EXPLORIMMO
63,209,1.000000,"Marclay's Immobilier vous propose un local commercial de 40 m² de plain pied situé dan

Nous avons pu predire et affecter nos données de test dans des clusters dont les annonces conrespondaient ou ils ont été considérés comme etant des annonces uniques

## **2ème Modèle**

###  **2) Methode de similarité pour traiter les descriptions**


Nous utilisons la similarité cosinus pour mesurer la similarité entre 2 paires des descriptions (la représentation vectorielle de la description). Nous allons utiliser cette methode et reprendre l'entraitement de notre modèle.

La méthode de similarité cosinus (cosine similarity) est couramment utilisée pour mesurer la similitude entre des textes, des documents ou des vecteurs de texte dans le traitement automatique du langage naturel (NLP). Elle repose sur une représentation vectorielle des textes, où chaque document est représenté sous forme d'un vecteur dans un espace multidimensionnel.

Pour appliquer la similarité cosinus, les textes sont tout d'abord représentés sous forme de vecteurs dans un espace vectoriel. Il existe différentes méthodes pour effectuer cette représentation, notamment la représentation en sac de mots (bag of words) ou des techniques de plongement de mots telles que Word2Vec, GloVe ou des modèles de langage pré-entraînés comme BERT. Chaque dimension de ces vecteurs correspond à un mot ou une caractéristique spécifique du texte.

Une fois que les textes sont représentés sous forme de vecteurs, vous obtenez deux vecteurs correspondant aux textes que vous souhaitez comparer.
La similarité cosinus mesure l'angle entre les deux vecteurs dans l'espace vectoriel.

 Plus la similarité cosinus est proche de 1, plus les textes sont similaires, car leurs vecteurs pointent dans des directions similaires. Plus elle est proche de -1, plus les textes sont différents. Une similarité cosinus proche de 0 signifie que les textes n'ont pas de similarité particulière.


In [ ]:
from sentence_transformers import SentenceTransformer, util
import umap


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('dangvantuan/sentence-camembert-large')
model.to(device)
if device == 'cuda':
    model = model.half()

max_desc_len = 500

description_embeddings = df['DESCRIPTION'].apply(lambda x: f"query: {str(x)[:max_desc_len]}").tolist()
description_embeddings = model.encode(description_embeddings, show_progress_bar=True, convert_to_tensor=False)

# calculer du cosine de similarités entre les pairs des descriptions
description_embeddings_cos_sim = util.pytorch_cos_sim(description_embeddings, description_embeddings)

df_cos_sim = pd.DataFrame(description_embeddings_cos_sim)


Batches:   0%|          | 0/68 [00:00<?, ?it/s]

In [ ]:
df_cos_sim.shape

(2164, 2164)

Le desavantage de cette technique est qu'il exige beaucoup de resources et nous crée plus des variables car nous calculons la cosine de similarité entre chaque valeur et tous les autres deux à deux cad si nous avons N valeurs dans une variable, le cosine de similarité va nous donner N variables

### **Train et split pour le 2ème modele**

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
df_with_all_features = pd.concat([df_scaled, df_cos_sim], axis=1)
df_with_all_features['CRAWL_SOURCE_original'] = df['CRAWL_SOURCE']
df_with_all_features = pd.concat([df_with_all_features, df['DESCRIPTION']], axis=1)
X_train_full, X_test_full = train_test_split(df_with_all_features, test_size=test_size, random_state=42)
X_train, X_test = X_train_full.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore'), X_test_full.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore')


In [ ]:
#affichage de la forme de nos données après division
X_train.shape, X_test.shape, X_train_full.shape, X_test_full.shape

((1731, 2195), (433, 2195), (1731, 2197), (433, 2197))

### Entrainement du 2e modèle

In [ ]:
import hdbscan
seed_everything(42)
clusterer = hdbscan.HDBSCAN(min_cluster_size=2,
                            min_samples=1, # min_cluster_size by default
                            alpha=5.0, # 1.0 by default
                            # cluster_selection_epsilon=3, # 0.0 by default
                            prediction_data=True
                            )
clusterer.fit(X_train)


HDBSCAN(alpha=5.0, min_cluster_size=2, min_samples=1, prediction_data=True)

In [ ]:
# umap dimensionality reduction
seed_everything(42)
reducer = umap.UMAP(n_neighbors=6, n_components=2)
X_train_pca = reducer.fit_transform(X_train)


In [ ]:
# plot clusters

X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION'].str[:600]
X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION_wrapped'].str.wrap(100)
X_train_full['DESCRIPTION_wrapped'] = X_train_full['DESCRIPTION_wrapped'].apply(lambda x: str(x).replace('\n', '<br>'))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X_train_pca[:, 0],
    y=X_train_pca[:, 1],
    mode='markers',
    marker=dict(
        size=5,
        color=clusterer.labels_,

    ),
    text=X_train.index,
    customdata=X_train_full['DESCRIPTION_wrapped'],
    hovertemplate='Description: %{customdata}<br>Index: %{text}<br>Cluster: %{marker.color}<extra></extra>'
))

# increase size of the plot
fig.update_layout(
    title='Clusters',
    xaxis_title='PCA 1',
    yaxis_title='PCA 2',
    legend_title='Cluster',
    font=dict(
        family='Courier New, monospace',
        size=18,
        color='#7f7f7f'
    ),
    width=1200,
    height=800
)
X_train_full.drop('DESCRIPTION_wrapped', axis=1, inplace=True)

fig.show()

### Ajout de la colonne de regroupement pour le 2è modèle

In [ ]:

def add_cluster_column(clusterer, X_train_):
    X_train = copy.deepcopy(X_train_).reset_index(drop=True)
    labels = clusterer.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Number of outliers/unique ads: %d' % list(labels).count(-1))
    # print max amount of clusters, mean amount of clusters and min amount of clusters
    print('Max cluster: %d' % np.sort(np.unique(labels, return_counts=True)[1])[::-1][1]) # ignore -1
    print('Mean cluster: %d' % np.unique(labels, return_counts=True)[1].mean())
    print('Min cluster: %d' % np.unique(labels, return_counts=True)[1].min())


    # undo scaling
    _X_train = pd.DataFrame(scaler_transformer.named_transformers_['Scaler'].inverse_transform(X_train[numeric_cols]), columns=numeric_cols)
    X_train[numeric_cols] = _X_train[numeric_cols]
    # add cluster column
    X_train['CLUSTER'] = labels
    # make a column that contains distance from the cluster centroid. -1 distance means that the point is an outlier
    X_train['CLUSTER_DISTANCE'] = clusterer.outlier_scores_
    return X_train


X_train_full_clustered = add_cluster_column(clusterer, X_train_full)

Estimated number of clusters: 52
Number of outliers/unique ads: 1566
Max cluster: 12
Mean cluster: 32
Min cluster: 2


Nous voyons que notre 2ème modèle a reussi de faire 52 groupements des annonces similaires et à classer 1566 annonces comme étant uniques.

Nous pouvons aussi remarquer que le plus gros groupe a 12 annonces similaires, le plus petit groupe en a 2 et la moyenne des annonces dans les groupes est de 32


### Affichage de nos resultat de regroupement

In [ ]:
# Affichage de 5 premiers cluster ayant plus des annonces
print(X_train_full_clustered['CLUSTER'].value_counts()[:5])

-1     1566
 39      12
 50      10
 36       8
 26       7
Name: CLUSTER, dtype: int64


In [ ]:
# Affichage par cluster
display(X_train_full_clustered.sort_values(by='CLUSTER', ascending=False)[['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head(50))

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
575,51,0.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",687000.0,61.43,SE_LOGER
667,51,0.878252,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",704000.0,60.72,SE_LOGER
1086,51,0.752538,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",626000.0,60.72,SE_LOGER
1331,51,0.878252,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",742000.0,65.53,SE_LOGER
441,51,0.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",687000.0,61.85,SE_LOGER
207,51,0.752538,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",661000.0,64.05,SE_LOGER
257,50,0.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio 

Nous remarquons que notre modèle à tres bien pu detecter les annonces similaires. Surtout quand dans le groupe il y'a moins de 3 annonces similaires mais en cas de plus de 3 le modèle n'arrive plus à bien sonder la difference les descriptions de ses annonces sont presque similaires

In [ ]:
# Affichage du groupe le plus gros
X_train_full_clustered[X_train_full_clustered['CLUSTER'].value_counts().index[1] == X_train_full_clustered['CLUSTER']][['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head(50)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
204,39,0.000000,"92300 LEVALLOIS PERRET - VRAI COUP DE COEUR : Je vous propose ce superbe restaurant sans extraction avec 22 places dont une dizaine de couverts en terrasse. Ce bien a une superficie totale de plus de 35 m2. Déco refait à neuf. Aucun travaux à prévoir! Loyer mensuel 1540 euros TTC. Idéal vente à emporter et livraisons. A découvrir rapidement! Cession à 69000 euros honoraires inclus à la charge du vendeur. Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n°199606",69000.0,35.0,EXPLORIMMO
319,39,0.003838,"92300 LEVALLOIS PERRET - VRAI COUP DE COEUR : Je vous propose ce restaurant de restauration rapide avec extraction dans un quartier d'affaires. Ce restaurant d'une superficie de 50m2, dispose de 6/8 places assises et terrasse. Matériel très récent. Bail échéance en 2028. Aucun travaux à prévoir! Loyer mensuel 1990 euros. A découvrir! Cession à 79900 euros honoraires inclus à la charge du vendeur. Affaire à saisir rapidement! Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 187446",79900.0,50.0,EXPLORIMMO
359,39,0.000000,"92300 LEVALLOIS PERRET - RESTAURATION RAPIDE SANS EXTRACTION : Je vous propose ce restaurant de restauration rapide idéalement situé dans un quartier de bureaux, écoles et commerçants, avec 10/12 places intérieur et d'une superficie de 25 m2. Loyer mensuel 940 euros. Cession à 99200 euros honoraires inclus à la charge du vendeur. Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 188597",99200.0,25.0,BIEN_ICI
503,39,0.000016,"92300 LEVALLOIS PERRET - RESTAURATION RAPIDE SANS EXTRACTION - Je vous propose ce restaurant de restauration rapide sandwicherie et saladerie idéalement situé dans un quartier de bureaux, écoles et commerçant. Superficie de 20 m2 avec 6 places debout. Pas d'extraction. Loyer mensuel très faible à 998 euros. Cession à 127900 euros honoraires inclus à la charge du vendeur. Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 188156",127900.0,20.0,BIEN_ICI
698,39,0.003959,"92300 LEVALLOIS PERRET - RESTAURATION RAPIDE SANS EXTRACTION : Je vous propose ce restaurant de restauration rapide idéalement situé dans un quartier de bureaux, écoles et commerçants, avec 10/12 places intérieur et d'une superficie de 25 m2. Loyer mensuel 940 euros. Cession à 99200 euros honoraires inclus à la charge du vendeur. Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 188597 Réf. annonce : 2807FR188597B-FRZ",99200.0,25.0,LOGIC_IMMO
713,39,0.000016,"92300 LEVALLOIS PERRET - RESTAURATION RAPIDE SANS EXTRACTION - Je vous propose ce restaurant de restauration rapide sandwicherie et saladerie idéalement situé dans un quartier de bureaux, écoles et commerçant. Superficie de 20 m2 avec 6 places debout. Pas d'extraction. Loyer mensuel très faible à 998 euros. Cession à 127900 euros honoraires inclus à la charge du vendeur. Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 188156",127900.0,20.0,EXPLORIMMO
769,39,0.000000,"92300 LEVALLOIS PERRET - RESTAURATION RAPIDE SANS EXTRACTION : Je vous propose ce restaurant de restauration rapide idéalement situé dans un quartier de b

Nous pouvons aussi remarqué qu'avec le gros cluster notre 2e modèle à reussi à dectecter les annonces similaire venant de plusieurs platforme differentes mais aaussi failli en ajoutant les annonces qui sont presque similaires mais qui ne sont pas les memes.

In [ ]:
# Affichage du groupe le plus petit
X_train_full_clustered[X_train_full_clustered['CLUSTER'].value_counts().index[-1] == X_train_full_clustered['CLUSTER']][['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'SURFACE', 'CRAWL_SOURCE_original']].head(50)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,SURFACE,CRAWL_SOURCE_original
1462,15,0.0,"RUE RIVAY - Au sein d'une copropriété bien tenue, au 3ème sous-sol, un BOX mesurant 2,49m largeur, 4,6m longueur, 1,93 hauteur. A visiter sans tarder !!!!",44000.0,12.0,BIEN_ICI
1605,15,0.0,"RUE RIVAY - Au sein d'une copropriété bien tenue, au 3ème sous-sol, un BOX mesurant 2,49m largeur, 4,6m longueur, 1,93 hauteur. A visiter sans tarder !!!!",44000.0,12.0,EXPLORIMMO


Pour le groupe le plus petit, notre cluster a correctement détecter ces deux annonces similaires venant de deux platformes differentes.

### **Prediction de regroupement les données de test**

In [ ]:
def pred_cluster(clusterer, df, scaled_cols):
    df = copy.deepcopy(df).reset_index(drop=True)
    labels, strengths = hdbscan.approximate_predict(clusterer, df.drop(col_del+['CRAWL_SOURCE_original'], axis=1, errors='ignore'))

    # undo scaling
    _df = pd.DataFrame(
    scaler_transformer.transformers_[0][1].inverse_transform(df[numeric_cols]),
    columns=numeric_cols
)

    df[scaled_cols] = _df[scaled_cols] # update X_train with unscaled values
    # add cluster column
    df['CLUSTER'] = labels
    # make a column that contains distance from the cluster centroid. -1 distance means that the point is an outlier
    df['CLUSTER_DISTANCE'] = strengths
    return df
X_test_full_clustered = pred_cluster(clusterer, X_test_full, numeric_cols)

### Affichage de resultats de la prediction du 2è modèle avec nos données de test

In [ ]:
# Affichage de resulats de la prediction
X_test_full_clustered.sort_values(by='CLUSTER', ascending=False)[['CLUSTER', 'CLUSTER_DISTANCE', 'DESCRIPTION', 'PRICE', 'CRAWL_SOURCE_original']].head(40)

,CLUSTER,CLUSTER_DISTANCE,DESCRIPTION,PRICE,CRAWL_SOURCE_original
286,50,1.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",483000.0,SE_LOGER
188,50,0.495147,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",576000.0,SE_LOGER
348,50,1.000000,"Une adresse de choix pour devenir propriétaire. La station de métro, le lycée et les commerces sont accessibles à pied dans un rayon de 500 m*. Vous apprécierez la qualité résidentielle de la commune et choisirez parmi une typologie variée d'appartements du studio au 5 pièces duplex. Les espaces intérieurs sont pour la plupart prolongés d'agréables extérieurs, balcons, terrasses ou jardins privatifs. Un coeur d'îlot paysagrer et des espaces communs végétalisés avec cheminements piétons desservent les différents halls d'entrée sécurisés. *Source: Google Maps.",477000.0,SE_LOGER
167,48,0.771735,EVOLIS vous propose une surface de bureaux rénovée avec terrasse technique dans un immeuble situé à toute proximité de la Porte de Champerret. Les locaux sont très lumineux et climatisés.,11001.0,SE_LOGER
322,39,1.000000,"92300 LEVALLOIS - BELLE AFFAIRE - Je vous propose un restaurant d'angle avec licence IV, offrant une surface de plus de 100 m² ouverte sur une vitrine de 10 mètres, ainsi qu'un sous-sol de 30 m2... Idéalement situé dans le quartier résidentiel et commerçant de Levallois-Perret, cet établissement de bonne réputation exploite actuellement près de 60 couverts, ainsi que 10 places en terrasse. Emplacement centre ville, exceptionnel et rare. Possibilité de transformation en brasserie. Excellent chiffre d'affaires. Loyer 2.890 euros. Prix 690.000 euros honoraires charge vendeur . Contactez Dominique GIRAUD-DESERT au 06 62 23 01 49 agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES. CPI : 75012016000006389. Mandat n° 204762",690000.0,EXPLORIMMO
212,39,0.662596,"92300 LEVALLOIS PERRET - VRAI COUP DE COEUR : Je vous propose ce restaurant de restauration rapide avec extraction dans un quartier d'affaires. Ce restaurant d'une superficie de 50m2, dispose de 6/8 places assises et terrasse. Matériel très récent. Bail échéance en 2028. Aucun travaux à prévoir! Loyer mensuel 1990 euros. A découvrir! Cession à 79900 euros honoraires inclus à la charge du vendeur. Affaire à saisir rapidement! Contactez Mme Frédérique Rézé au 06 72 37 96 11, spécialisée en cession de fonds de commerce et agissant en tant que chargée d'affaires indépendante auprès de la SAS PROPRIETES PRIVEES - Mandat n° 187446",79900.0,BIEN_ICI
206,39,0.662603,"92300 LEVALLOIS PERRET - VRAI COUP DE COEUR : Je vous propose ce superbe restaurant sans extraction avec 22 places dont une dizaine de couverts en terrasse. Ce bien a une superficie totale de plus de 35 m2. Déco refait à neuf. Aucun travaux à prévoir! Loyer mensuel 1540 euros TTC. Idéal vente à emporter et livraisons. A découvrir rapidement! Cession à 69000 euros honoraires inclus à la charge du vendeur. Contactez Mme 

Nous avons pu predire et affecter nos données de test dans des clusters dont les annonces conrespondaient ou ils ont été considérés comme etant des annonces uniques.

## **Conclusion et Perspectives d'Amélioration**

Le premier modèle, qui utilise l'encodage par "embedding" exclusivement pour la colonne "DESCRIPTION", a réussi à regrouper efficacement des annonces similaires, bien que parfois il ait associé des annonces qui ne l'étaient pas, mais dont les descriptions étaient très semblables. Selon nos résultats, c'est le modèle qui s'en est le mieux sorti.

Le deuxième modèle, qui se base sur la similarité cosinus, a bien regroupé les annonces similaires, mais a échoué à détecter d'autres annonces similaires s'il n'y avait pas de proximité évidente dans les descriptions. Il a également eu du mal à distinguer des annonces avec des descriptions lexicalement proches mais non identiques.

Pour une utilisation fréquente, le premier modèle s'avère plus adapté si l'objectif est de prédire le groupe de nouvelles annonces inconnues, car il n'y a pas besoin de réentraîner l'encodage par "embedding", et le modèle généralisera mieux vers de nouvelles données.

Le deuxième modèle est plus pertinent si l'objectif est de détecter les similarités dans les descriptions des données connues pour lesquelles la similarité cosinus a été calculée. Toutefois, il n'est pas adéquat pour des données inconnues, car il ne sera pas en mesure de détecter des similitudes approximatives.

Ainsi, le premier modèle est préférable pour des prédictions, en raison de sa capacité à généraliser, tandis que le deuxième modèle est utile pour analyser des données connues.

**Perspectives d'Amélioration :**

- L'exploration de la variable IMAGE, qui n'a pas été abordée dans ce travail, pourrait s'avérer très utile. Si nous pouvons établir des liens de similarité et détecter des images similaires, cela pourrait être une extension précieuse de cette analyse.

- Un traitement plus approfondi de la colonne DESCRIPTION est recommandé. Il est évident que des erreurs d'orthographe subsistent, probablement dues à la récupération des données par l'API. Par exemple, des mots mal orthographiés, des caractères inattendus comme des caractères spéciaux mélangés avec des mots, et d'autres petites erreurs de structure nécessitent une correction.

Je suis ouvert aux appréciations et aux critiques constructives, car je suis conscient que mes compétences dans ce domaine sont en constante évolution. J'aspire à travailler sur des projets stimulants afin de continuer à progresser.

Je reste à votre disposition pour discuter en détail de ce travail si vous le souhaitez.


In [3]:
! pip install nbconvert >/dev/null 2>/dev/nul
! apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic >/dev/null 2>/dev/nul

In [4]:
!jupyter nbconvert --to pdf "/content/drive/MyDrive/Colab Notebooks/Ads_clustering_Joel_KabanguKabobo.ipynb" >/dev/null 2>/dev/nul